# Prototype of API connection

https://medium.com/@bhaveshpatelaus/gtfs-realtime-vehicle-positions-using-python-and-databricks-tfnsw-a33b98f22e97

In [1]:
import os
from dotenv import load_dotenv
import requests
from pathlib import Path
import zipfile
import pandas as pd

from google.transit import gtfs_realtime_pb2
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
from google.protobuf.json_format import MessageToJson

from protobuf_to_dict import protobuf_to_dict

from data import data

load_dotenv()

True

In [2]:
FILENAME_SCHEDULE = 'gtfs.zip'

In [3]:
app_name = os.getenv("APP_NAME")
api_key = os.getenv("API_KEY")

In [4]:
BASE_URL = "https://api.transport.nsw.gov.au"
BUS_POSITION_URI = f"{BASE_URL}/v1/gtfs/vehiclepos/buses"
BUS_SCHEDULE_URI = f"{BASE_URL}/v1/gtfs/schedule/buses"
FERRY_POSITION = f"{BASE_URL}/v1/gtfs/historical"

In [9]:
headers = {
    "Authorization": f"apikey {api_key}"
}
request_details = dict(
    headers=headers,
    stream=True
)
if cert:=os.getenv("CERT", None):
    request_details['verify'] = cert

## Extract schedules

In [11]:
response = requests.get(BUS_SCHEDULE_URI, **request_details)
response

<Response [200]>

In [12]:
zip_path = Path(data.path / FILENAME_SCHEDULE)

In [13]:
with open(zip_path, "wb") as f:
    f.write(response.content)

In [68]:
with zipfile.ZipFile(zip_path) as zip:
    print(zip.namelist())

['agency.txt', 'calendar.txt', 'calendar_dates.txt', 'notes.txt', 'routes.txt', 'shapes.txt', 'stops.txt', 'stop_times.txt', 'trips.txt']


In [73]:
with zipfile.ZipFile(zip_path) as z:
   with z.open("shapes.txt") as f:
      train = pd.read_csv(f)
      print(train.head())    # print the first 5 rows

   shape_id  shape_pt_lat  shape_pt_lon  shape_pt_sequence  \
0       177    -33.418715    151.341307                  1   
1       177    -33.418708    151.341302                  2   
2       177    -33.418686    151.341128                  3   
3       177    -33.418567    151.340324                  4   
4       177    -33.418519    151.339836                  5   

   shape_dist_traveled  
0                    0  
1                    9  
2                   25  
3                  101  
4                  147  


## Realtime locations

In [14]:
response = requests.get(BUS_POSITION_URI, **request_details)
response

<Response [200]>

In [15]:
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

199176

In [16]:
print(len(feed.entity))
feed.entity[0]

1114


id: "33553_26249868_2436_600_1"
vehicle {
  trip {
    trip_id: "1954191"
    route_id: "2436_600"
    start_time: "19:40:00"
    start_date: "20230911"
    schedule_relationship: SCHEDULED
  }
  vehicle {
    id: "33553_26249868_2436_600_1"
  }
  position {
    latitude: -33.718853
    longitude: 151.107452
    bearing: 44
    speed: 15.3
  }
  timestamp: 1694428312
  congestion_level: RUNNING_SMOOTHLY
  occupancy_status: MANY_SEATS_AVAILABLE
}

In [17]:
feed.ParseFromString(response.content)
#for entity in feed.entity:
#    if entity.HasField('vehicle'):
#        print(entity.vehicle)

dict_obj = protobuf_to_dict(feed)
dict_obj['entity'][0]

{'id': '33553_26249868_2436_600_1',
 'vehicle': {'trip': {'trip_id': '1954191',
   'start_time': '19:40:00',
   'start_date': '20230911',
   'schedule_relationship': 0,
   'route_id': '2436_600'},
  'position': {'latitude': -33.71885299682617,
   'longitude': 151.10745239257812,
   'bearing': 44.0,
   'speed': 15.300000190734863},
  'timestamp': 1694428312,
  'congestion_level': 1,
  'vehicle': {'id': '33553_26249868_2436_600_1'},
  'occupancy_status': 1}}

In [20]:
collector = []
counter=0
for block in dict_obj["entity"]:
    counter += 1
    row = {}
    row["id"] = block["id"]
    trip = block["vehicle"]["trip"]
    row["trip_id"] = trip.get("trip_id","")
    row["route_id"] = trip.get("route_id","")
    row["schedule_relationship"] = trip.get("schedule_relationship","")
    if "position" in block['vehicle']:
        row["lat"] = block["vehicle"]["position"].get("latitude","")
        row["lon"] = block["vehicle"]["position"].get("longitude","")
        row["bearing"] = block["vehicle"]["position"].get("bearing","")
        row["speed"] = block["vehicle"]["position"].get("speed","")
    else:
        row['lat'], row['lon'], row['bearing'], row['speed'] = None, None
    
    row["timestamp"] = block["vehicle"].get("timestamp","")
    row["congestion_level"] = block["vehicle"].get("congestion_level","")
    row["stop_id"] = block["vehicle"].get("stop_id","")
    row["vehicle_id"] = block["vehicle"]["vehicle"].get("id","")
    row["label"] = block["vehicle"]["vehicle"].get("label","")
    collector.append(row)
    df = pd.DataFrame(collector)

In [24]:
df.head()

,id,trip_id,route_id,schedule_relationship,latitude,longitude,bearing,speed,timestamp,congestion_level,stop_id,vehicle_id,label
0,33553_26249868_2436_600_1,1954191,2436_600,0,-33.718853,151.107452,44.0,15.3,1694428312,1,,33553_26249868_2436_600_1,
1,43280_230178305_2509_370_1,1861739,2509_370,0,-33.903030,151.179520,329.0,8.9,1694428320,1,,43280_230178305_2509_370_1,
2,33553_26249463_2436_711_1,1712973,2436_711,0,-33.772385,150.935974,92.0,9.4,1694428316,1,,33553_26249463_2436_711_1,
3,33553_26249024_2436_601_1,1939023,2436_601,0,-33.737682,150.983673,341.0,12.3,1694428315,1,,33553_26249024_2436_601_1,
4,43332_207638338_2503_T80_1,1966181,2503_T80,0,-33.818394,151.009079,342.0,0.0,1694428130,0,,43332_207638338_2503_T80_1,


In [30]:
from visualisations import maps
maps.position_map(df.rename(columns={'latitude': 'lat', 'longitude': 'lon'}))

,id,trip_id,route_id,schedule_relationship,lat,lon,bearing,speed,timestamp,congestion_level,stop_id,vehicle_id,label
0,33553_26249868_2436_600_1,1954191,2436_600,0,-33.718853,151.107452,44.0,15.3,1694428312,1,,33553_26249868_2436_600_1,
1,43280_230178305_2509_370_1,1861739,2509_370,0,-33.903030,151.179520,329.0,8.9,1694428320,1,,43280_230178305_2509_370_1,
2,33553_26249463_2436_711_1,1712973,2436_711,0,-33.772385,150.935974,92.0,9.4,1694428316,1,,33553_26249463_2436_711_1,
3,33553_26249024_2436_601_1,1939023,2436_601,0,-33.737682,150.983673,341.0,12.3,1694428315,1,,33553_26249024_2436_601_1,
4,43332_207638338_2503_T80_1,1966181,2503_T80,0,-33.818394,151.009079,342.0,0.0,1694428130,0,,43332_207638338_2503_T80_1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,8896_135525760039_2433_732_1,1338060,2433_732,0,-33.693413,150.902237,347.0,6.9,1694428311,1,,8896_135525760039_2433_732_1,
1110,8896_135505510083_2433_672_1,1365902,2433_672,0,-33.606930,150.820465,326.0,4.0,1694428321,0,,8896_135505510083_2433_672_1,
1111,8896_135512470082_2433_783_1,1376700,2433_783,0,-33.729820,150.694656,197.0,8.6,1694428322,1,,8896_135512470082_2433_783_1,
1112,8896_135507830082_2433_776_1,1726213,2433_776,0,-33.798058,150.792038,312.0,10.4,1694428321,0,,8896_135507830082_2433_776_1,
